In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,ConfusionMatrixDisplay, precision_recall_fscore_support, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# also set a random state
rs = 123

## **1. Exploratory Data Analysis(EDA) and Feature Engineering**
Before we get to the model implementation, it is essential to examine the dataset and carefully select the features that will serve as inputs for the model.


### Load and explore the dataset


First, let's load the dataset as a `Pandas` dataframe and conduct some basic EDA tasks on it.


In [ ]:
# Load the dataset
dataset_path = "https://raw.githubusercontent.com/andresmorenoviteri/ML-models/main/telecom_customer_churn.csv"
churn_df = pd.read_csv(dataset_path)

And, let's quickly check its column types.


In [ ]:
churn_df.____

Print the first ten customers items:


In [ ]:
churn_df.head(____)

Let's drop `Churn Category` and `Churn Reason`:

In [ ]:
churn_df = churn_df.drop([____], axis=1)

Obtain descriptive statistics:

In [ ]:
churn_df.____

Next, let's check the target variable in the `Customer Status` column to see the label values and their distribution.

In [ ]:
churn_df[____].____()

## **2. Feature Engineering**

Keep only the rows that don't contain `Joined` in `Customer Status`:

In [ ]:
# Type your code here
churn_df = ____

Check that `Customer Status` only has 2 types:

In [ ]:
churn_df['Customer Status']._____

Create an **object_df** where the only data types are 'object': 

In [ ]:
object_df = churn_df.select_dtypes(include=[____])
object_df.head()

Remove columns that are probably not helpful like: `Customer ID`, `Gender`, `Married`, `City`

In [ ]:
object_df = ____
object_df.head()

Find the rows that have nan values:

In [ ]:
# get only the columns with nan values 
object_df.loc[:, object_df.isna().any()].columns

For objects, replace nan values with mode

In [ ]:
#create list from column names
null_objects = [____]

# create for-loop to replace with mode
for col in null_objects:
    object_df[col].fillna(object_df[col].____[0], inplace=True)


Create a **targets** variable with `Customer Status` and **object_df** without `Customer Status`

In [ ]:
targets = ____
object_df = _____

Encode the names to numbers for our model:

In [ ]:
encoded_object_df = pd.get_dummies(object_df, columns=object_df.columns.to_list(), drop_first=True)
encoded_object_df[encoded_object_df.columns.to_list()] = encoded_object_df[encoded_object_df.columns.to_list()].astype(int)
encoded_object_df.head()

Create an **churn_df_numeric** where the only data types are 'number': 

In [ ]:
churn_df_numeric = ____

In [ ]:
# get only the columns with nan values 
churn_df_numeric.____

In [ ]:
# replace nan values with average
null_numeric = [____]

for col in null_numeric:
    churn_df_numeric[col].fillna(____.mean(), inplace=True)


In [ ]:
# concatenate the two dataframes
clean_churn_data = pd.concat([____, ____], axis=1)
clean_churn_data.head()

Create a barplot to see how the labels are distributed:

In [ ]:
clean_churn_data['Customer Status'].value_counts().____.____(color=[____])

Let's process the raw dataset and construct input data `X` and label/output `y` for logistic regression model training.

In [ ]:
x_raw = ____
y_raw = ____
encoder = LabelEncoder()
y = encoder.fit_transform(y_raw)

In [ ]:
np.unique(y, return_counts=True)

All feature columns are now numeric so we just need to scale them. Here we use the `MinMaxScaler` provided by `sklearn` for scaling.


In [ ]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

In [ ]:
# Scaling the raw input features
x = scaler.fit_transform(____)

Let's check the scaled feature value range:


In [ ]:
print(f"The range of feature inputs are within {x.min()} to {x.max()}")

## **3. Train logistic regression models**

First, let's split the dataset into a training and a testing dataset. Training dataset will be used to train and (maybe) tune models, and testing dataset will be used to evaluate the models. Note that you may also split the training dataset into train and validation sets where the validation dataset is only used to tune the model and to set the model parameters.


In [ ]:
# First, let's split the training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(____, ____, ____ stratify=y, random_state = ____)

Let's look at the shapes of the split datasets:


In [ ]:
print(f"Training dataset shape, x_train: {x_train.shape}, y_train: {y_train.shape}")

In [ ]:
print(f"Testing dataset shape, x_test: {x_test.shape}, y_test: {y_test.shape}")

OK, now we have the training and testing datasets ready, let's start the model training task.


We first define a `sklearn.linear_model.LogisticRegression` model with the following arguments, you can check the comment for each argument for what it means.


In [ ]:
# L2 penalty to shrink coefficients without removing any features from the model
# penalty= 'l2'
# Use lbfgs for L2 penalty and multinomial classes
# solver = 'lbfgs'
# Max iteration = 1000
max_iter = 1000

In [ ]:
# Define a logistic regression model with above arguments
l2_model = LogisticRegression(random_state=rs, max_iter=max_iter)

Let's train the model with training input data `X_train` and labels `y_train`:


In [ ]:
l2_model.____

In [ ]:
l2_preds = ____

Because we may need to evaluate the model multiple times with different model hyper parameters, here we define an utility method to take the ground truths `y_test` and the predictions `preds`, and return a Python `dict` with `accuracy`, `recall`, `precision`, and `f1score`.


In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp)
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
evaluate_metrics(y_test, l2_preds)

In [ ]:
cf = confusion_matrix(y_test, l2_preds, normalize='true')
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=l2_model.classes_)
disp.plot()
plt.show()

As we can see from  the above evaluation results, the logistic regression model has relatively good performance on this binomial classification task. The overall accuracy is around `0.86` and the f1score is around `0.82`. Note that for `recall`, `precision`, and `f1score`, we output the values for each class to see how the model performs on an individual class. And, we can see from the results, the recall for `class=0` (Churned) is only ok. This is actually a common problem called imbalanced classification challenge. 


Next, let's try defining another logistic regression model with l1 penality this time, to see if our classification performance would be improved.


In [ ]:
# L1 penalty to shrink coefficients without removing any features from the model
penalty= 'l1'
# Use saga for L1 penalty and multinomial classes
solver = 'saga'
# Max iteration = 1000
max_iter = 1000

Then we define another logistic regression model with above arguments using l1 penality and related solver.


In [ ]:
# Define a logistic regression model with above arguments
l1_model = LogisticRegression(random_state=____, penalty=____, solver=____, max_iter = ____)

We can start to train the new `l1_model` with the new taining dataset.


In [ ]:
l1_model.____

And, make predictions using the input in the test dataset.


In [ ]:
l1_preds = ____

We can also check the class probability distribution using the `predict_proba` function. For example, we want to see the probabilities of belonging to each class for the first instance in the test dataset:


In [ ]:
odd_ratios = l1_model.predict_proba(x_test[:1, :])[0]
odd_ratios

We can see that  Class 1 has the largest probability 0.96. As such, the model prediction for this instance will be class `1` and this is the same as the `predict` method.


In [ ]:
l1_model.predict(x_test[:1, :])[0]

Given the true labels (`y_test`) and predictions, we can evaluate the model performance by calling the utility `evaluate_metrics`  method.


In [ ]:
evaluate_metrics(____)

### Confusion Matrix


We can also plot the confusion matrix based on the true labels and predictions using the `confusion_matrix` method provided by `sklearn`,


In [ ]:
cf = confusion_matrix(y_test, l1_preds, normalize='true')

and easily visualize it using a heatmap method provided by `seaborn`.


In [ ]:
sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=l1_model.classes_)
disp.plot()
plt.show()

### Interpret logistic regression models


One way to interpret logistic regression models is by analyzing feature coefficients. Although it may not be as effective as the regular linear regression models because the logistic regression model has a sigmoid function, we can still get a sense for the importance or impact of each feature.  


We can check the coefficients for logistic regression model using its `coef_` attribute:


In [ ]:
l1_model.coef_

The `coef_` is a coefficients list with three elements, one element is the actual coefficent for class 0, 1, 2. To better analyze the coefficients, let's use three utility methods to sort and visualize them.


In [ ]:
# Extract and sort feature coefficients
def get_feature_coefs(regression_model, label_index, columns):
    coef_dict = {}
    for coef, feat in zip(regression_model.coef_[label_index, :], columns):
        if abs(coef) >= 0.01:
            coef_dict[feat] = coef
    # Sort coefficients
    coef_dict = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1])}
    return coef_dict

# Generate bar colors based on if value is negative or positive
def get_bar_colors(values):
    color_vals = []
    for val in values:
        if val <= 0:
            color_vals.append('r')
        else:
            color_vals.append('g')
    return color_vals

# Visualize coefficients
def visualize_coefs(coef_dict):
    features = list(coef_dict.keys())
    values = list(coef_dict.values())
    y_pos = np.arange(len(features))
    color_vals = get_bar_colors(values)
    plt.rcdefaults()
    fig, ax = plt.subplots()
    ax.barh(y_pos, values, align='center', color=color_vals)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(features)
    # labels read top-to-bottom
    ax.invert_yaxis()  
    ax.set_xlabel('Feature Coefficients')
    ax.set_title('')
    plt.show()

In [ ]:
feature_cols = clean_churn_data.columns[clean_churn_data.columns != 'Customer Status']

Then, let's visualize the sorted coefficient for class 0, the `Churned` class: 


In [ ]:
# Get the coefficents for Class 0, Stayed
coef_dict = get_feature_coefs(l1_model, 0, feature_cols)

In [ ]:
visualize_coefs(coef_dict)

As we can see, Tenure in Months, Number of Dependents, Number of Referrals, Contract, and Phone Service huave high positive coefficients. Total Charges, Monthly Charges, Maried and Age will most likely not influence a churn.

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=200,       # number of trees
    max_depth=1,            # max depth of each tree
    learning_rate=0.1,      # step size shrinkage
    random_state=42,
    eval_metric='logloss'   # evaluation metric
)
xgb_model.fit(x_train, y_train)

# predict
y_pred = xgb_model.predict(x_test)

# evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

sns.set_context('talk')
disp = ConfusionMatrixDisplay(confusion_matrix=cf,display_labels=xgb_model.classes_)
disp.plot()
plt.show()